# Spring 2022
# CPSC 585 Project 4
## Raymond Carpio
## Yu Pan
## Sijie Shang
## John Tu

In [ ]:
# Import the following modules over
import numpy as np
import tensorflow as tf
import os
import time

# Try to open the text file if it exists.
input_file = "rings.txt"
corpus_raw = open(input_file, "r", encoding="utf-8").read()

# Print out the sample text file.
#print(corpus_raw)

In [ ]:
# Obtain the list of characters included in the raw text.
# Be sure to use set() to filter out duplicates.
characters = sorted(list(set(corpus_raw)))

# Print out the list of characters found in the raw text.
print(characters)

# Also obtain the total length of the text and the characters.
print("Total length of text: ", len(corpus_raw))
print("Total number of characters found: ", len(characters))

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Ó', 'á', 'â', 'ä', 'é', 'ë', 'í', 'ó', 'ú', 'û', '–']
Total length of text:  1021058
Total number of characters found:  90


In [ ]:
# Print out the first 500 characters of the text.
print(corpus_raw[:500])

Three Rings for the Elven-kings under the sky,
               Seven for the Dwarf-lords in their halls of stone,
            Nine for Mortal Men doomed to die,
              One for the Dark Lord on his dark throne
           In the Land of Mordor where the Shadows lie.
               One Ring to rule them all, One Ring to find them,
               One Ring to bring them all and in the darkness bind them
           In the Land of Mordor where the Shadows lie.
           
FOREWORD

This tale grew


In [ ]:
# Vectorize the text by converting the string into a numerical form.
sample_text = ['abcdefghij', 'wxyz']
num_chars = tf.strings.unicode_split(sample_text, input_encoding="UTF-8")
num_chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [ ]:
id_from_characters = tf.keras.layers.StringLookup(vocabulary=list(characters))
ids = id_from_characters(num_chars)
ids

<tf.RaggedTensor [[54, 55, 56, 57, 58, 59, 60, 61, 62, 63], [76, 77, 78, 79]]>

In [ ]:
characters_from_id = tf.keras.layers.StringLookup(vocabulary=id_from_characters.get_vocabulary(), invert=True)
characters = characters_from_id(ids)
characters

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g', b'h', b'i', b'j'],
 [b'w', b'x', b'y', b'z']]>

In [ ]:
# Convert the corresponding string ids back into text form.
def text_from_id(ids):
    return tf.strings.reduce_join(characters_from_id(ids), axis=-1)

In [ ]:
all_text_id = id_from_characters(tf.strings.unicode_split(corpus_raw, 'UTF-8'))
all_text_id

<tf.Tensor: shape=(1021058,), dtype=int64, numpy=array([45, 61, 71, ..., 32, 10,  1])>

In [ ]:
# Try to print out the first 25 characters from raw text via tensor slices.
id_dataset = tf.data.Dataset.from_tensor_slices(all_text_id)
for curr_id in id_dataset.take(25):
    print(characters_from_id(curr_id).numpy().decode('utf-8'))

T
h
r
e
e
 
R
i
n
g
s
 
f
o
r
 
t
h
e
 
E
l
v
e
n


In [ ]:
# Now build a sequence of the first 150 characters.
sequence_len = 150
each_example = len(corpus_raw)
sequences = id_dataset.batch(sequence_len+1, drop_remainder=True)

for curr_seq in sequences.take(1):
    print(characters_from_id(curr_seq))

tf.Tensor(
[b'T' b'h' b'r' b'e' b'e' b' ' b'R' b'i' b'n' b'g' b's' b' ' b'f' b'o'
 b'r' b' ' b't' b'h' b'e' b' ' b'E' b'l' b'v' b'e' b'n' b'-' b'k' b'i'
 b'n' b'g' b's' b' ' b'u' b'n' b'd' b'e' b'r' b' ' b't' b'h' b'e' b' '
 b's' b'k' b'y' b',' b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'S' b'e' b'v' b'e' b'n' b' ' b'f' b'o'
 b'r' b' ' b't' b'h' b'e' b' ' b'D' b'w' b'a' b'r' b'f' b'-' b'l' b'o'
 b'r' b'd' b's' b' ' b'i' b'n' b' ' b't' b'h' b'e' b'i' b'r' b' ' b'h'
 b'a' b'l' b'l' b's' b' ' b'o' b'f' b' ' b's' b't' b'o' b'n' b'e' b','
 b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b'N'
 b'i' b'n' b'e' b' ' b'f' b'o' b'r' b' ' b'M' b'o' b'r' b't' b'a' b'l'
 b' ' b'M' b'e' b'n' b' ' b'd' b'o' b'o' b'm' b'e' b'd'], shape=(151,), dtype=string)


In [ ]:
for curr_seq in sequences.take(5):
    print(text_from_id(curr_seq).numpy())

b'Three Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed'
b' to die,\n              One for the Dark Lord on his dark throne\n           In the Land of Mordor where the Shadows lie.\n               One Ring to rule'
b' them all, One Ring to find them,\n               One Ring to bring them all and in the darkness bind them\n           In the Land of Mordor where the Sh'
b'adows lie.\n           \nFOREWORD\n\nThis tale grew in the telling, until it became a history of the Great War of the Ring and included many glimpses of th'
b'e yet more ancient history that preceded it. It was begun soon after _The Hobbit_ was written and before its publication in 1937; but I did not go on w'


In [ ]:
def split_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
text_dataset = sequences.map(split_target)
for input_text, target_text in text_dataset.take(1):
    print("Input: ", text_from_id(input_text).numpy())
    print("Target: ", text_from_id(target_text).numpy())

Input:  b'Three Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doome'
Target:  b'hree Rings for the Elven-kings under the sky,\n               Seven for the Dwarf-lords in their halls of stone,\n            Nine for Mortal Men doomed'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    text_dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 150), dtype=tf.int64, name=None), TensorSpec(shape=(64, 150), dtype=tf.int64, name=None))>

In [ ]:
vocab = sorted(set(corpus_raw))
vocab_size = len(vocab)
embedding_dimension=256
rnn_units=1024

In [ ]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(id_from_characters.get_vocabulary()),
    embedding_dim=embedding_dimension,
    rnn_units=rnn_units)

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 150, 91) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  23296     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  93275     
                                                                 
Total params: 4,054,875
Trainable params: 4,054,875
Non-trainable params: 0
_________________________________________________________________


In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([39,  5,  9, 44,  4, 42, 47, 57, 38,  4, 17, 17, 42, 42, 41, 47, 49,
       66, 26, 41, 52,  5, 60, 74, 14, 33,  6,  7, 65, 82, 16, 80, 65, 42,
       85, 51, 49, 28, 84, 52, 83, 52, 78,  0,  8, 82, 41, 66, 41, 77, 59,
       50, 89, 25, 21, 85, 21, 36, 40, 88,  7, 33,  3, 54, 61, 11, 25, 73,
       88, 47, 16, 89, 29, 44, 74, 62, 77, 13, 55, 17, 26, 68, 16, 81, 65,
       29, 72, 34, 52, 48, 74, 43, 38, 21, 74, 39, 88, 24, 62, 17, 12, 25,
       22, 49, 71, 14,  5,  4, 35, 54, 14, 81, 69,  0, 23, 79, 19, 40, 15,
       19, 26, 57,  2,  8, 51, 85, 81, 16, 71, 36, 29, 26, 83, 46, 58, 70,
       36, 19, 86, 31, 75, 65, 50, 36, 16, 49, 66,  9, 81, 69])

In [ ]:
print("Input:\n", text_from_id(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_id(sampled_indices).numpy())

Input:
 b'e Hill, carrying hundreds of polite variations on _Thank you, I shall certainly come._\n     A notice appeared on the gate at Bag End: NO ADMITTANCE EX'

Next Char Predictions:
 b'N\'-S"QVdM"55QQPVXmAP_\'gu2H()l\xc3\xa24\xc3\x93lQ\xc3\xabZXC\xc3\xa9_\xc3\xa4_y[UNK],\xc3\xa2PmPxfY\xc3\xbb?9\xc3\xab9KO\xc3\xba)H!ah/?t\xc3\xbaV4\xc3\xbbDSuix1b5Ao4\xc3\xa1lDsI_WuRM9uN\xc3\xba=i50?:Xr2\'"Ja2\xc3\xa1p[UNK];z7O37Ad ,Z\xc3\xab\xc3\xa14rKDA\xc3\xa4UeqK7\xc3\xadFvlYK4Xm-\xc3\xa1p'


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)
tf.exp(example_batch_mean_loss).numpy()

Prediction shape:  (64, 150, 91)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.5094204, shape=(), dtype=float32)


90.86913

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=100, callbacks=[checkpoint_callback])

Epoch 1/100
105/105 [==============================] - 29s 204ms/step - loss: 2.8814
Epoch 2/100
105/105 [==============================] - 23s 199ms/step - loss: 2.0925
Epoch 3/100
105/105 [==============================] - 23s 200ms/step - loss: 1.8001
Epoch 4/100
105/105 [==============================] - 23s 203ms/step - loss: 1.5896
Epoch 5/100
105/105 [==============================] - 23s 199ms/step - loss: 1.4448
Epoch 6/100
105/105 [==============================] - 23s 202ms/step - loss: 1.3481
Epoch 7/100
105/105 [==============================] - 23s 202ms/step - loss: 1.2804
Epoch 8/100
105/105 [==============================] - 23s 200ms/step - loss: 1.2275
Epoch 9/100
105/105 [==============================] - 23s 202ms/step - loss: 1.1846
Epoch 10/100
105/105 [==============================] - 23s 200ms/step - loss: 1.1485
Epoch 11/100
105/105 [==============================] - 23s 201ms/step - loss: 1.1131
Epoch 12/100
105/105 [==============================] - 23s 202

In [ ]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, characters_from_id, id_from_characters)

In [ ]:
start = time.time()
states = None
next_char = tf.constant(['The'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Then we must be expected, for the strangers when Gandalf had as they saw right and dark. Dragons had a mind toel far away both breathless. I think we ought to make out about Strider, who could have known but this path that we must deed in tall blaze boulders and he learned all this look. Fear was found in the fire and bright;
           not matter may be found, that it is the party who once wore; but that will not like you good advice Meriadoc and I think; but I don't think would private my strength.'
     `What avel amory glimpses and or a few turn of memory.'
     'Behind the Stoor – when neighrood of Sauron's name must be back to the road we could do. I could wish to meet. Very much like Busibel, and took the truth, he was quite by adays.'
     'Very well,' said Boromir guard. 'I know that way, I feel any handing?'
     'He did,' said Sam looking down, or of Gandalf's. Undeed we can rest in these parts at times, except a few stunting scent. A chair of Things returned now above them.